In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from datetime import datetime

import googletrans

import requests
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np


import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob

In [2]:
def url_request(url):
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome()
    driver.get(url)    
    return driver

In [3]:
driver = url_request('https://www.rt.com')

## Function to get DF

In [4]:
def key_word(word):
    
    cookies= driver.find_element(By.PARTIAL_LINK_TEXT,'Accept cookies').click()
    
    
    search_bar = driver.find_element(By.CLASS_NAME, "js-search-input")
    search_bar.send_keys(word)
    search_bar.send_keys(Keys.RETURN)

    lst_link=[]
    try:

        while len(lst_link)< 10:
            lnks       = driver.find_elements(By.TAG_NAME,"a")
            lst_rt     = [lnk.get_attribute('href') for lnk in lnks]
            lst_rt_1   = [url for url in lst_rt if 'https://www.rt.com/news/5' in str(url)]
            lst_link   = [i for i in lst_rt_1]

            more_button= driver.find_element(By.ID,'listingBtn')
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)",more_button)
            more_button.click()

            lst_link=list(set(lst_link))

            list_soup = []
            for url in lst_link:
                html  = requests.get(url)
                soup  = BeautifulSoup(html.content, "html.parser")
                list_soup.append(soup)
            art_list = []  
            for soup in list_soup:
                article = soup.getText()
                article = article[article.find('Home'):]
                article = article[:article.find('You can share this story on social media:')]
                article = article.replace('\xa0','').replace('\n','').replace('READ MORE','').replace('/','').replace('HomeWorld News','')
                art_list.append(article)

            day_list=[]
            month_list=[]
            year_list=[]
            year_control=[]

            for soup in list_soup:
                date_str = soup.find('span', {'class': 'date'}).text.strip()
                day, month, year, time = date_str.split()
                day = day.zfill(2)
                month = month.zfill(2).replace(',','')
                year= year

                day_list.append(day)
                month_list.append(month)
                year_list.append(year)
                

        dict_={
            'link': lst_link,
            'article': art_list,
            'year' : year_list,
            'month': month_list,
            'day': day_list
        }
        df= pd.DataFrame(dict_)
        return df
    
    except:
        return df

In [5]:
df_US  = key_word('US')

In [6]:
df_USA = key_word('USA')

In [7]:
df_united_states = key_word('united states')

In [8]:
df_us= pd.concat([df_US,df_USA,df_united_states])

In [10]:
df_us.drop_duplicates(inplace=True)

In [ ]:
df_us.reset_index(inplace=True).drop(columns=['index'],inplace=True)